In [3]:
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.9 MB/s eta 0:00:00ta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [2]:
def extract_docs():
    from datasets import load_dataset
    xsum = load_dataset("EdinburghNLP/xsum")  # :contentReference[oaicite:1]{index=1}
    docs = {ex["id"]: ex["document"] for split in xsum.values()  # :contentReference[oaicite:2]{index=2}
            for ex in split}
    return docs
import pandas as pd
factual_df = pd.read_csv("../Data/factuality_annotations_xsum_summaries.csv")

docs = extract_docs()

/home/alireza/miniconda3/envs/NLP_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def extract_entities(text: str):
    """
    Returns the list of entity strings found in text.
    """
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

In [4]:
# def extract_svo_triples(text: str):
#     """
#     Returns list of (subject, verb, object) triples from text.
#     """
#     doc = nlp(text)
#     triples = []
#     for token in doc:
#         # look for subjects of verbs
#         if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
#             subj = token.text
#             verb = token.head.lemma_
#             # find direct objects of that verb
#             objs = [child.text for child in token.head.children
#                     if child.dep_ in ("dobj", "obj", "attr")]
#             for obj in objs:
#                 triples.append((subj, verb, obj))
#     return triples


In [5]:

# def extract_svo_triples(text: str):
#     """
#     Enhanced S–V–O extractor:
#       • Iterates over all VERB tokens (including conj children)
#       • Finds subjects via nsubj / nsubjpass
#       • Finds objects via dobj, pobj, attr, or prep→pobj
#       • Ignores pronoun subjects/objects
#     """
#     doc = nlp(text)
#     triples = []

#     for token in doc:
#         # consider any verb or its conjuncts
#         if token.pos_ != "VERB":
#             continue

#         # gather subjects
#         subjects = [
#             child for child in token.children
#             if child.dep_ in ("nsubj", "nsubjpass") and child.pos_ != "PRON"
#         ]
#         if not subjects:
#             continue

#         # gather direct objects and attribute objects
#         objects = [
#             child for child in token.children
#             if child.dep_ in ("dobj", "attr") and child.pos_ != "PRON"
#         ]

#         # gather objects via prepositions (prep → pobj)
#         for prep in [c for c in token.children if c.dep_ == "prep"]:
#             for pobj in [c2 for c2 in prep.children if c2.dep_ == "pobj" and c2.pos_ != "PRON"]:
#                 objects.append(pobj)

#         # if no direct or prep objects, skip
#         if not objects:
#             continue

#         # record every combination
#         for subj in subjects:
#             for obj in objects:
#                 triples.append((subj.text, token.lemma_, obj.text))

#     return triples


# src = docs[str(factual_df.loc[10, "bbcid"])]

# for sent in nlp(src).sents:
#     print(sent.text)
#     print("→", extract_svo_triples(sent.text))
#     print()


# # src   = docs[str(factual_df['bbcid'][10])]
# # summ  = factual_df['summary'][10]
# # print(src)
# # extract_svo_triples(src)

In [35]:
import spacy

class Phrases():
    def __init__(self, sentence):
       self.nlp = spacy.load('en_core_web_sm')
       self.sentence = str(sentence)
       self.doc = self.nlp(self.sentence)
       self.sequence = 0
       self.svos = []

    def merge_phrases(self):
        with self.doc.retokenize() as retokenizer:
            for np in list(self.doc.noun_chunks):
                    attrs = {
                        "tag": np.root.tag_,
                        "lemma": np.root.lemma_,
                        "ent_type": np.root.ent_type_,
                    }
                    retokenizer.merge(np, attrs=attrs)
        return self.doc

    def merge_punct(self):
        spans = []
        for word in self.doc[:-1]:
            if word.is_punct or not word.nbor(1).is_punct:
                continue
            start = word.i
            end = word.i + 1
            while end < len(self.doc) and self.doc[end].is_punct:
                end += 1
            span = self.doc[start:end]
            spans.append((span, word.tag_, word.lemma_, word.ent_type_))
        with self.doc.retokenize() as retokenizer:
            for span, tag, lemma, ent_type in spans:
                attrs = {"tag": tag, "lemma": lemma, "ent_type": ent_type}
                retokenizer.merge(span, attrs=attrs)
        return self.doc

    def is_passive(self, tokens):
        for tok in tokens:
            if tok.dep_ == "auxpass":
                return True
        return False

    def _is_verb(self, token):
        return token.dep_ in ["ROOT", "xcomp", "appos", "advcl", "ccomp", "conj"] and token.tag_ in ["VB", "VBZ", "VBD", "VBN", "VBG", "VBP"]

    def find_verbs(self, tokens):
        verbs = [tok for tok in tokens if self._is_verb(tok)]
        return verbs

    def get_all_subs(self, v):
        #get all subjects
        subs = [tok for tok in v.lefts if tok.dep_ in ["ROOT", "nsubj", "nsubjpass"] and tok.tag_ in ["NN" , "NNS", "NNP"]]
        if len(subs) == 0:
            #get all subjects from the left of verb ("nsubj" <= "preconj" <= VERB)
            subs = [tok for tok in v.lefts if tok.dep_ in ["preconj"]]
            for sub in subs:
                rights = list(sub.rights)
                right_dependency = [tok.lower_ for tok in rights]
                if len(right_dependency) > 0:
                    subs = right_dependency[0]
        return subs

    def get_all_objs(self, v, is_pas):
        #get list the right of dependency with VERB (VERB => "dobj" or "pobj")
        rights = list(v.rights)
        objs = [tok for tok in rights if tok.dep_ in ["dobj", "dative", "attr", "oprd", "pobj"] or (is_pas and tok.dep_ == 'pobj')]
        #get all objects from the right of dependency (VERB => "dobj" or "pobj")
        for obj in objs:
            #on the right of dependency, you can get objects from prepositions (VERB => "dobj" => "prep" => "pobj")
            rights = list(obj.rights) 
            objs.extend(self._get_objs_from_prepositions(rights, is_pas))
        return v, objs

    def _get_objs_from_prepositions(self, deps, is_pas):
        objs = []
        for dep in deps:
            if dep.pos_ == "ADP" and (dep.dep_ == "prep" or (is_pas and dep.dep_ == "agent")):
                objs.extend([tok for tok in dep.rights if tok.dep_  in ["dobj", "dative", "attr", "oprd", "pobj"] or (tok.pos_ == "PRON" and tok.lower_ == "me") or (is_pas and tok.dep_ == 'pobj')])
        return objs

    def main_get_to_pobj(self, verb):
        for tok in verb.rights:
            if tok.dep_ == "prep" and tok.lower_ == "to":
                for child in tok.children:
                    if child.dep_ == "pobj":
                        return child.text
        return None 

    def get_svo(self, sentence):
        doc = self.nlp(sentence)
        doc = self.merge_phrases()
        doc = self.merge_phrases()

        #check passive and active sentence
        is_pas = self.is_passive(doc)

        #find the main verb and child of a verb
        verbs = self.find_verbs(doc) 

        #more than verb
        for verb in verbs:
            self.sequence += 1

            #find the subject with the main verb
            subject = self.get_all_subs(verb)

            #find the object with the main verb                
            verb, obj = self.get_all_objs(verb, is_pas)

            #find prepositional modifier with the main verb  
            to_pobj = self.main_get_to_pobj(verb)

            #You can continue create method for extract word ...

            #finally, we can find prepositional modifier with the main verb
            if to_pobj is not None:
                #result SVO
                self.svos.append((self.sequence, subject, verb, obj, to_pobj))
            else:
                #result SVO
                self.svos.append((self.sequence, subject, verb, obj, ""))
        
        return self.svos

In [38]:
import spacy

# Load spaCy model once at the module level
nlp = spacy.load('en_core_web_sm')

def merge_phrases(doc):
    with doc.retokenize() as retokenizer:
        for np in list(doc.noun_chunks):
            attrs = {
                "tag": np.root.tag_,
                "lemma": np.root.lemma_,
                "ent_type": np.root.ent_type_,
            }
            retokenizer.merge(np, attrs=attrs)
    return doc

def merge_punct(doc):
    spans = []
    for word in doc[:-1]:
        if word.is_punct or not word.nbor(1).is_punct:
            continue
        start = word.i
        end = word.i + 1
        while end < len(doc) and doc[end].is_punct:
            end += 1
        span = doc[start:end]
        spans.append((span, word.tag_, word.lemma_, word.ent_type_))
    with doc.retokenize() as retokenizer:
        for span, tag, lemma, ent_type in spans:
            attrs = {"tag": tag, "lemma": lemma, "ent_type": ent_type}
            retokenizer.merge(span, attrs=attrs)
    return doc

def is_passive(doc):
    return any(tok.dep_ == "auxpass" for tok in doc)

def is_verb(tok):
    return tok.dep_ in ["ROOT", "xcomp", "appos", "advcl", "ccomp", "conj"] and tok.tag_ in ["VB", "VBZ", "VBD", "VBN", "VBG", "VBP"]

def find_verbs(doc):
    return [tok for tok in doc if is_verb(tok)]

def get_all_subs(verb):
    subs = [tok for tok in verb.lefts if tok.dep_ in ["ROOT", "nsubj", "nsubjpass"] and tok.tag_ in ["NN", "NNS", "NNP"]]
    if not subs:
        subs = [tok for tok in verb.lefts if tok.dep_ == "preconj"]
        for sub in subs:
            rights = list(sub.rights)
            if rights:
                return [rights[0]]
    return subs

def _get_objs_from_prepositions(deps, is_pas):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and (dep.dep_ == "prep" or (is_pas and dep.dep_ == "agent")):
            objs.extend([tok for tok in dep.rights if tok.dep_ in ["dobj", "dative", "attr", "oprd", "pobj"]
                         or (tok.pos_ == "PRON" and tok.lower_ == "me")
                         or (is_pas and tok.dep_ == 'pobj')])
    return objs

def get_all_objs(verb, is_pas):
    rights = list(verb.rights)
    objs = [tok for tok in rights if tok.dep_ in ["dobj", "dative", "attr", "oprd", "pobj"] or (is_pas and tok.dep_ == 'pobj')]
    for obj in objs:
        rights = list(obj.rights)
        objs.extend(_get_objs_from_prepositions(rights, is_pas))
    return objs

def main_get_to_pobj(verb):
    for tok in verb.rights:
        if tok.dep_ == "prep" and tok.lower_ == "to":
            for child in tok.children:
                if child.dep_ == "pobj":
                    return child.text
    return None

def extract_svo(sentence):
    doc = nlp(sentence)
    doc = merge_phrases(doc)
    doc = merge_phrases(doc)  # twice, as in original logic

    is_pas = is_passive(doc)
    verbs = find_verbs(doc)

    svos = []
    sequence = 0

    for verb in verbs:
        sequence += 1
        subs = get_all_subs(verb)
        objs = get_all_objs(verb, is_pas)
        to_pobj = main_get_to_pobj(verb)

        # convert tokens to text
        sub_text = [tok.text for tok in subs]
        obj_text = [tok.text for tok in objs]
        verb_text = verb.lemma_  # canonical verb form

        svos.append((sequence, sub_text, verb_text, obj_text, to_pobj or ""))

    return svos


In [41]:
src = docs[str(factual_df.loc[10, "bbcid"])]
print(src)

result = extract_svo( src)
for r in result:
    print(f"[{r[0]}] SUBJECT: {r[1]} | VERB: {r[2]} | OBJECT: {r[3]} | TO_POBJ: {r[4]}")


Former Dons midfielder Sheerin, 39, has been player-manager at the Red Lichties since 2010 and replaces Neil Cooper at Pittodrie.
Arbroath were relegated to Scottish League Two after finishing season 2013-14 bottom of League One.
Aberdeen chief executive Duncan Fraser said boss Derek McInnes chose Sheerin after "a comprehensive process".
Speaking on the club's website, Fraser added: "Despite wishing to retain Paul's services, Arbroath chairman John Christison was good to deal with and completely appreciated Paul's desire to come back to Aberdeen."
Sheerin played for several Scottish clubs and also featured for Östersunds in Sweden before moving into coaching. He won a Scotland Under-21 cap in in the mid-1990s.
Sheerin guided the Arbroath to their first national trophy in their history by winning the Scottish Third Division title and promotion in 2011.
[1] SUBJECT: ['Former Dons midfielder Sheerin'] | VERB: be | OBJECT: ['player-manager', 'the Red Lichties'] | TO_POBJ: 
[2] SUBJECT: [] 

In [ ]:
test.get_svo('ali went to the bathroom.')

[(1, [ali], went, [], 'the bathroom')]

In [ ]:
for seq, sub, verb, obj, to_pobj in svos:
    sub_text = [tok.text if hasattr(tok, 'text') else str(tok) for tok in sub]
    verb_text = verb.lemma_ if hasattr(verb, 'lemma_') else str(verb)
    obj_text = [tok.text if hasattr(tok, 'text') else str(tok) for tok in obj]
    print(f"[{seq}] SUBJECT: {sub_text} | VERB: {verb_text} | OBJECT: {obj_text} | TO_POBJ: {to_pobj}")


In [11]:

# object and subject constants
OBJECT_DEPS = {"dobj", "dative", "attr", "oprd"}
SUBJECT_DEPS = {"nsubj", "nsubjpass", "csubj", "agent", "expl"}
# tags that define wether the word is wh-
WH_WORDS = {"WP", "WP$", "WRB"}

# extract the subject, object and verb from the input
def extract_svo(doc):
    sub = []
    at = []
    ve = []
    for token in doc:
        # is this a verb?
        if token.pos_ == "VERB":
            ve.append(token.text)
        # is this the object?
        if token.dep_ in OBJECT_DEPS or token.head.dep_ in OBJECT_DEPS:
            at.append(token.text)
        # is this the subject?
        if token.dep_ in SUBJECT_DEPS or token.head.dep_ in SUBJECT_DEPS:
            sub.append(token.text)
    return " ".join(sub).strip().lower(), " ".join(ve).strip().lower(), " ".join(at).strip().lower()

# wether the doc is a question, as well as the wh-word if any
def is_question(doc):
    # is the first token a verb?
    if len(doc) > 0 and doc[0].pos_ == "VERB":
        return True, ""
    # go over all words
    for token in doc:
        # is it a wh- word?
        if token.tag_ in WH_WORDS:
            return True, token.text.lower()
    return False, ""



src = docs[str(factual_df.loc[10, "bbcid"])]

for sent in nlp(src).sents:
    print(sent.text)
    for token in sent:
        print("Token {} POS: {}, dep: {}".format(token.text, token.pos_, token.dep_))
    print("→", extract_svo(sent))
    subject, verb, attribute =  extract_svo(sent)
    print("svo:, subject: {}, verb: {}, attribute: {}, question: {}, wh_word: {}".format(subject, verb, attribute, question, wh_word))
    print()

# doc   = docs[str(factual_df['bbcid'][10])]
# get the input information
# subject, verb, attribute = extract_svo(doc)
question, wh_word = "is_question(doc)" , "is_question(doc)"


Former Dons midfielder Sheerin, 39, has been player-manager at the Red Lichties since 2010 and replaces Neil Cooper at Pittodrie.

Token Former POS: ADJ, dep: amod
Token Dons POS: PROPN, dep: compound
Token midfielder POS: NOUN, dep: compound
Token Sheerin POS: PROPN, dep: nsubj
Token , POS: PUNCT, dep: punct
Token 39 POS: NUM, dep: appos
Token , POS: PUNCT, dep: punct
Token has POS: AUX, dep: aux
Token been POS: AUX, dep: ROOT
Token player POS: NOUN, dep: compound
Token - POS: PUNCT, dep: punct
Token manager POS: NOUN, dep: attr
Token at POS: ADP, dep: prep
Token the POS: DET, dep: det
Token Red POS: PROPN, dep: compound
Token Lichties POS: PROPN, dep: pobj
Token since POS: SCONJ, dep: prep
Token 2010 POS: NUM, dep: pobj
Token and POS: CCONJ, dep: cc
Token replaces POS: VERB, dep: conj
Token Neil POS: PROPN, dep: compound
Token Cooper POS: PROPN, dep: dobj
Token at POS: ADP, dep: prep
Token Pittodrie POS: PROPN, dep: pobj
Token . POS: PUNCT, dep: punct
Token 
 POS: SPACE, dep: dep
→ (

In [ ]:
# import spacy

# nlp = spacy.load("en_core_web_sm")

# def extract_svo_triples(text: str):
#     doc = nlp(text)
#     svos = []

#     for sent in doc.sents:
#         # 1) Identify ROOT verbs (VERB or AUX) and build subject map
#         roots = [t for t in sent if t.dep_=="ROOT" and t.pos_ in ("VERB","AUX")]
#         subj_map = {}
#         for root in roots:
#             subs = [c for c in root.children
#                     if c.dep_ in ("nsubj","nsubjpass") and c.pos_!="PRON"]
#             subj_map[root] = subs

#         # 2) Collect all verbs = roots + their conjuncts
#         verbs = []
#         for root in roots:
#             verbs.append(root)
#             verbs.extend([c for c in root.conjuncts if c.pos_ in ("VERB","AUX")])

#         # 3) Extract per-verb SVO
#         for verb in verbs:
#             # a) subjects: direct or inherited
#             subs = [c for c in verb.children
#                     if c.dep_ in ("nsubj","nsubjpass") and c.pos_!="PRON"]
#             if not subs and verb.dep_=="conj":
#                 # inherit from its head root
#                 head = verb.head
#                 subs = subj_map.get(head, [])

#             if not subs:
#                 continue

#             # b) objects: attr/dobj/dative + prep→pobj
#             objs = []
#             for c in verb.children:
#                 if c.dep_ in ("dobj","attr","dative") and c.pos_!="PRON":
#                     objs.append(c)
#                 if c.dep_=="prep":
#                     for pobj in c.children:
#                         if pobj.dep_=="pobj" and pobj.pos_!="PRON":
#                             objs.append(pobj)
#             if not objs:
#                 continue

#             # c) expand spans & record triples
#             for subj in subs:
#                 subj_text = subj.text  # head only
#                 for obj in objs:
#                     # drop pure numbers
#                     if obj.pos_ == "NUM":
#                         continue
#                     # expand object to phrase
#                     span = sent[obj.left_edge.i - sent.start : obj.right_edge.i - sent.start + 1]
#                     obj_text = span.text
#                     svos.append((subj_text, verb.lemma_, obj_text))

#     return svos

# # Quick sanity test
# example = (
#     "Former Dons midfielder Sheerin, 39, has been player-manager at the Red Lichties "
#     "since 2010 and replaces Neil Cooper at Pittodrie."
# )
# for sent in nlp(example).sents:
#     print("SENTENCE:", sent.text)
#     print("→", extract_svo_triples(sent.text))
#     print()


SENTENCE: Former Dons midfielder Sheerin, 39, has been player-manager at the Red Lichties since 2010 and replaces Neil Cooper at Pittodrie.
→ [('Sheerin', 'be', 'player-manager at the Red Lichties'), ('Sheerin', 'replace', 'Neil Cooper'), ('Sheerin', 'replace', 'Pittodrie')]



In [31]:
import spacy

nlp = spacy.load("en_core_web_sm")


def extract_svo_triples(text: str):
    """
    - Sentence‐level SVO
    - Verbs: ROOT, ccomp, conj (VERB or AUX)
    - Subjects: nsubj/nsubjpass (drop pronouns)
    - Objects: dobj/attr/dative + first pobj under prep + ccomp handled separately
    - Expand every object via its subtree
    - Drop objects whose text starts with "their "
    - Dedupe via a set
    """
    doc = nlp(text)
    triples = set()

    for sent in doc.sents:
        # 1) Collect candidate verbs
        verbs = [t for t in sent
                 if t.pos_ in ("VERB","AUX") and t.dep_ in ("ROOT","ccomp","conj")]
        # plus conjuncts of those
        for v in list(verbs):
            verbs.extend([c for c in v.conjuncts if c.pos_ in ("VERB","AUX")])

        for verb in verbs:
            # 2) Subjects
            subs = [c for c in verb.children
                    if c.dep_ in ("nsubj","nsubjpass") and c.pos_!="PRON"]
            # inherit for conj w/o own subs
            if not subs and verb.dep_=="conj":
                subs = [c for c in verb.head.children
                        if c.dep_ in ("nsubj","nsubjpass") and c.pos_!="PRON"]
            if not subs:
                continue

            # 3) Objects
            objs = []
            for c in verb.children:
                if c.dep_ in ("dobj","attr","dative"):
                    objs.append(c)
                if c.dep_ == "prep":
                    # first pobj under this prep
                    for pobj in c.children:
                        if pobj.dep_ == "pobj":
                            objs.append(pobj)
                            break
                # handle clausal complements as separate SVOs
                if c.dep_ == "ccomp":
                    nested = extract_svo_triples(c.text)
                    for (ns,nv,no) in nested:
                        for sub in subs:
                            triples.add((sub.text, verb.lemma_, f"{nv} {no}"))

            if not objs:
                continue

            # 4) Build and filter triples
            for sub in subs:
                subj_text = sub.text
                for obj in objs:
                    if obj.pos_ in ("PRON","NUM"):
                        continue
                    # full subtree span
                    toks = list(obj.subtree)
                    span = sent[toks[0].i - sent.start : toks[-1].i - sent.start + 1]
                    obj_text = span.text
                    if obj_text.lower().startswith("their "):
                        continue
                    triples.add((subj_text, verb.lemma_, obj_text))

    return list(triples)

# Sanity‐check on your example:
src = docs[str(factual_df.loc[10, "bbcid"])]
for sent in nlp(src).sents:
    print("SENTENCE:", sent.text)
    print("→", extract_svo_triples(sent.text))
    print()




# Quick sanity check on your example:
if __name__ == "__main__":
    src = docs[str(factual_df.loc[10, "bbcid"])]
    for sent in nlp(src).sents:
        print("SENTENCE:", sent.text)
        print("→", extract_svo_triples(sent.text))
        print()


SENTENCE: Former Dons midfielder Sheerin, 39, has been player-manager at the Red Lichties since 2010 and replaces Neil Cooper at Pittodrie.

→ [('Sheerin', 'replace', 'Pittodrie'), ('Sheerin', 'be', 'player-manager at the Red Lichties'), ('Sheerin', 'replace', 'Neil Cooper')]

SENTENCE: Arbroath were relegated to Scottish League Two after finishing season 2013-14 bottom of League One.

→ [('Arbroath', 'relegate', 'Scottish League')]

SENTENCE: Aberdeen chief executive Duncan Fraser said boss Derek McInnes chose Sheerin after "a comprehensive process".

→ [('McInnes', 'choose', 'a comprehensive process'), ('McInnes', 'choose', 'Sheerin')]

SENTENCE: Speaking on the club's website, Fraser added: "Despite wishing to retain Paul's services, Arbroath chairman John Christison was good to deal with and completely appreciated Paul's desire to come back to Aberdeen."
Sheerin played for several Scottish clubs and also featured for Östersunds in Sweden before moving into coaching.
→ [('Sheerin', 

In [ ]:
src = docs[str(factual_df.loc[10, "bbcid"])]

for sent in nlp(src).sents:
    print(sent.text)
    print("→", extract_svo_triples(sent.text))
    print()


In [6]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer("all-MiniLM-L6-v2")  # :contentReference[oaicite:3]{index=3}


/home/alireza/miniconda3/envs/NLP_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def match_entity(entity: str, source: str, threshold: float = 0.8) -> bool:
    # exact substring?
    if entity in source:
        return True
    # semantic match against each sentence
    sents = [sent.text for sent in nlp(source).sents]
    e_emb  = embedder.encode(entity, convert_to_tensor=True)
    s_embs = embedder.encode(sents,   convert_to_tensor=True)
    scores = util.cos_sim(e_emb, s_embs)
    return scores.max().item() >= threshold


In [10]:
def match_triple(triple: tuple[str,str,str], source_triples: list, threshold: float = 0.8) -> bool:
    # exact match?
    if triple in source_triples:
        return True
    # semantic: compare string form to each source triple
    txt   = " ".join(triple)
    src_txts = [" ".join(t) for t in source_triples]
    t_emb    = embedder.encode(txt,        convert_to_tensor=True)
    src_embs = embedder.encode(src_txts,   convert_to_tensor=True)
    scores   = util.cos_sim(t_emb, src_embs)
    return scores.max().item() >= threshold


In [11]:
def match_triple(triple: tuple[str,str,str], source_triples: list, threshold: float = 0.8) -> bool:
    # exact match?
    if triple in source_triples:
        return True
    # semantic: compare string form to each source triple
    txt   = " ".join(triple)
    src_txts = [" ".join(t) for t in source_triples]
    t_emb    = embedder.encode(txt,        convert_to_tensor=True)
    src_embs = embedder.encode(src_txts,   convert_to_tensor=True)
    scores   = util.cos_sim(t_emb, src_embs)
    return scores.max().item() >= threshold


In [23]:
import json

def compute_fact_score(summary: str, source: str, alpha: float = 0.5):
    # 1. extract
    ents_summ  = extract_entities(summary)
    ents_src   = extract_entities(source)
    triples_s  = extract_svo_triples(summary)
    triples_src= extract_svo_triples(source)

    # 2. match & collect unsupported
    sup_ents, unsup_ents = 0, []
    for e in ents_summ:
        if match_entity(e, source):
            sup_ents += 1
        else:
            unsup_ents.append(e)

    sup_trips, unsup_trips = 0, []
    for t in triples_s:
        if match_triple(t, triples_src):
            sup_trips += 1
        else:
            unsup_trips.append(t)

    # 3. precision scores
    ent_prec = sup_ents / len(ents_summ)     if ents_summ else 1.0
    tri_prec = sup_trips / len(triples_s)    if triples_s else 1.0

    # 4. combined FactScore
    fact_score = alpha * ent_prec + (1-alpha) * tri_prec

    # optional JSON error report
    error_report = {
        "entity_hallucinations": unsup_ents,
        "unsupported_triples":  unsup_trips
    }

    return fact_score, error_report

# Example usage
# factual_df = pd.read_csv("../Data/factuality_annotations_xsum_summaries.csv")

# docs = extract_docs()
src   = docs[str(factual_df['bbcid'][10])]
summ  = factual_df['summary'][10]

print(src)
print(summ)
score, report = compute_fact_score(summ, src, alpha=0.6)
print("FactScore:", score)
print("Errors:", json.dumps(report, indent=2))


Former Dons midfielder Sheerin, 39, has been player-manager at the Red Lichties since 2010 and replaces Neil Cooper at Pittodrie.
Arbroath were relegated to Scottish League Two after finishing season 2013-14 bottom of League One.
Aberdeen chief executive Duncan Fraser said boss Derek McInnes chose Sheerin after "a comprehensive process".
Speaking on the club's website, Fraser added: "Despite wishing to retain Paul's services, Arbroath chairman John Christison was good to deal with and completely appreciated Paul's desire to come back to Aberdeen."
Sheerin played for several Scottish clubs and also featured for Östersunds in Sweden before moving into coaching. He won a Scotland Under-21 cap in in the mid-1990s.
Sheerin guided the Arbroath to their first national trophy in their history by winning the Scottish Third Division title and promotion in 2011.
aberdeen have appointed arbroath boss paul sheerin as their new manager.
FactScore: 0.0
Errors: {
  "entity_hallucinations": [
    "aber